<a href="https://colab.research.google.com/github/rai8896/Deep_Learning_Lab/blob/main/rai8896/Deep_Learning_Lab/main/Experiment_4/cat_vs_dog4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
import os
import copy
import time
import itertools
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
import os
import zipfile
import shutil

# Clean old dataset if exists
shutil.rmtree("/kaggle/working/dataset", ignore_errors=True)

base_dir = "/kaggle/working/dataset"
os.makedirs(base_dir, exist_ok=True)

train_zip = "/kaggle/input/dogs-vs-cats/train.zip"

# Extract ZIP
with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall(base_dir)

# Paths
extracted_train = os.path.join(base_dir, "train")
cats_dir = os.path.join(extracted_train, "cats")
dogs_dir = os.path.join(extracted_train, "dogs")

os.makedirs(cats_dir, exist_ok=True)
os.makedirs(dogs_dir, exist_ok=True)

# Move ONLY image files (not folders)
for file in os.listdir(extracted_train):
    file_path = os.path.join(extracted_train, file)

    if os.path.isfile(file_path):
        if file.startswith("cat"):
            shutil.move(file_path, os.path.join(cats_dir, file))
        elif file.startswith("dog"):
            shutil.move(file_path, os.path.join(dogs_dir, file))

print("✅ Dataset ready at:", extracted_train)


✅ Dataset ready at: /kaggle/working/dataset/train


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

data_dir = "/kaggle/working/dataset/train"

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

dataset = datasets.ImageFolder(data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("Classes:", dataset.classes)
print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))


Classes: ['cats', 'dogs']
Train size: 20000
Val size: 5000


In [ ]:
import torch
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self, activation="relu"):
        super().__init__()

        if activation == "relu":
            self.act = nn.ReLU()
        elif activation == "tanh":
            self.act = nn.Tanh()
        else:
            self.act = nn.LeakyReLU(0.1)

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            self.act,
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            self.act,
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            self.act,
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*16*16, 256),
            self.act,
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x



In [ ]:
def initialize_weights(model, method="xavier"):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            if method == "xavier":
                nn.init.xavier_uniform_(m.weight)
            elif method == "kaiming":
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
            else:
                nn.init.normal_(m.weight, mean=0, std=0.02)


In [ ]:
import torch.optim as optim
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, optimizer, epochs=3):
    criterion = nn.CrossEntropyLoss()
    best_acc = 0
    best_weights = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)

        acc = correct.double() / len(train_loader.dataset)
        print(f"Epoch {epoch+1} | Loss: {running_loss:.3f} | Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_weights)
    return model, best_acc


In [ ]:
import itertools

activations = ["relu", "tanh", "leaky"]
inits = ["xavier", "kaiming", "random"]
optimizers_list = ["sgd", "adam", "rmsprop"]

results = {}

for act, init, opt in itertools.product(activations, inits, optimizers_list):

    print(f"\nRunning: {act} | {init} | {opt}")

    model = CustomCNN(act).to(device)
    initialize_weights(model, init)

    if opt == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif opt == "adam":
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)

    trained_model, acc = train_model(model, optimizer)

    key = f"{act}_{init}_{opt}"
    results[key] = acc.item()

    torch.save(trained_model.state_dict(), f"/kaggle/working/{key}.pth")

print("\nAll Results:", results)



Running: relu | xavier | sgd
Epoch 1 | Loss: 493.759 | Acc: 0.5053
Epoch 2 | Loss: 433.490 | Acc: 0.5079
Epoch 3 | Loss: 433.411 | Acc: 0.5015

Running: relu | xavier | adam
Epoch 1 | Loss: 658.858 | Acc: 0.5464
Epoch 2 | Loss: 402.849 | Acc: 0.6083
Epoch 3 | Loss: 381.759 | Acc: 0.6501

Running: relu | xavier | rmsprop
Epoch 1 | Loss: 1931.476 | Acc: 0.6005
Epoch 2 | Loss: 409.974 | Acc: 0.6466
Epoch 3 | Loss: 367.599 | Acc: 0.6854

Running: relu | kaiming | sgd
Epoch 1 | Loss: 457.370 | Acc: 0.5233
Epoch 2 | Loss: 432.209 | Acc: 0.5048
Epoch 3 | Loss: 432.251 | Acc: 0.5005

Running: relu | kaiming | adam
Epoch 1 | Loss: 627.449 | Acc: 0.5973
Epoch 2 | Loss: 372.432 | Acc: 0.6785
Epoch 3 | Loss: 350.933 | Acc: 0.7032

Running: relu | kaiming | rmsprop
Epoch 1 | Loss: 2005.067 | Acc: 0.5433
Epoch 2 | Loss: 414.099 | Acc: 0.5942
Epoch 3 | Loss: 372.135 | Acc: 0.6824

Running: relu | random | sgd
Epoch 1 | Loss: 444.629 | Acc: 0.5064
Epoch 2 | Loss: 433.825 | Acc: 0.4989
Epoch 3 | Loss:

In [ ]:
best_cnn = max(results, key=results.get)
print("Best CNN:", best_cnn, "Accuracy:", results[best_cnn])


Best CNN: leaky_xavier_adam Accuracy: 0.7623500000000001
